this notebook was used to perform the kolmogorov smirnov test experiment on all features of the NF datasets

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
from os import cpu_count
from math import floor
import pandas as pd
import numpy as np

In [4]:
datasets={}
for dirname, _, filenames in os.walk('/project_ghent/raman/netflow_datasets'):
    for filename in filenames:
        if filename.endswith("parquet"):
            datasets[filename.replace(".parquet","")]=pd.read_parquet(os.path.join(dirname, filename))

In [5]:
for k,v in datasets.items():
    print(k)
    print(v.Attack.unique())

NF-ToN-IoT-V2
['ransomware' 'Benign' 'xss' 'scanning' 'password' 'dos' 'ddos'
 'injection' 'mitm' 'backdoor']
NF-UNSW-NB15-V2
['Benign' 'Exploits' 'Generic' 'Fuzzers' 'Backdoor' 'DoS' 'Reconnaissance'
 'Shellcode' 'Worms' 'Analysis']
NF-BoT-IoT-V2
['DoS' 'DDoS' 'Reconnaissance' 'Benign' 'Theft']
NF-CSE-CIC-IDS2018-V2
['SSH-Bruteforce' 'Benign' 'DDoS attacks-LOIC-HTTP' 'DDOS attack-HOIC'
 'DoS attacks-Slowloris' 'DoS attacks-Hulk' 'FTP-BruteForce'
 'Infilteration' 'Bot' 'DoS attacks-GoldenEye' 'Brute Force -Web'
 'DoS attacks-SlowHTTPTest' 'SQL Injection' 'DDOS attack-LOIC-UDP'
 'Brute Force -XSS']


In [6]:
for ds,data in datasets.items():
    ddos=data[data['Attack'].str.contains("ddos",case=False)]
    ddos.Attack="DDoS"#uniform name
    dos=data[data['Attack'].str.contains("dos",case=False)]
    dos.Attack="DoS"#uniform name
    dos.drop(index=ddos.index,inplace=True)
    benign=data[data['Attack'] == 'Benign']
    
    data = pd.concat(objs=[ddos, dos, benign])
    
    # random forest classifier works internally with float32 so values too big will give errors
    data = data[data['SRC_TO_DST_SECOND_BYTES'] <= np.finfo(np.float32).max]
    data = data[data['DST_TO_SRC_SECOND_BYTES'] <= np.finfo(np.float32).max]
    
    data=data.drop(columns=['L4_SRC_PORT', 'L4_DST_PORT']) # dropping metadata

    
    datasets[ds]=data

/tmp/ipykernel_148/3237961987.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddos.Attack="DDoS"#uniform name
/tmp/ipykernel_148/3237961987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dos.Attack="DoS"#uniform name
/tmp/ipykernel_148/3237961987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dos.drop(index=dd

In [7]:
for k,v in datasets.items():
    print(k)
    print(v.Attack.value_counts())

NF-ToN-IoT-V2
Attack
Benign    3601194
DDoS      1746590
DoS        654359
Name: count, dtype: int64
NF-UNSW-NB15-V2
Attack
Benign    1911666
DoS          4172
Name: count, dtype: int64
NF-BoT-IoT-V2
Attack
DDoS      14280259
DoS       13645057
Benign      129437
Name: count, dtype: int64
NF-CSE-CIC-IDS2018-V2
Attack
Benign    15101526
DDoS       1276019
DoS         483999
Name: count, dtype: int64


In [8]:
features=data.columns[:-2]
features

Index(['PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE'],
      dtype='object')

In [18]:
from scipy.stats import kstest


if the p-value is below 5%: we reject the null hypothesis that the two distributions are the same, with 95% confidence.

### DoS

In [21]:
def evaluate_feature(feature):
    data= list(datasets.keys())
    result=[]
    for i in range(len(data)):
        for j in range(i+1,len(data)):
            # print(f"{data[i]} , {data[j]}")
            stat, p_value = kstest(datasets[data[i]][datasets[data[i]].Attack=="DoS"][feature], datasets[data[j]][datasets[data[j]].Attack=="DoS"][feature])
            # if(p_value>0.05):
            result.append([f"{data[i]} , {data[j]}", feature, stat,p_value])
    return result

In [22]:
from fastcore.parallel import *
from os import cpu_count

results = parallel(f=evaluate_feature, items=features, n_workers=cpu_count(), threadpool=True, progress=True)

In [23]:
results

(#39) [[['NF-ToN-IoT-V2 , NF-UNSW-NB15-V2', 'PROTOCOL', 0.7597017770218777, 0.0], ['NF-ToN-IoT-V2 , NF-BoT-IoT-V2', 'PROTOCOL', 0.9236369757090724, 0.0], ['NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2', 'PROTOCOL', 0.9313221030046198, 0.0], ['NF-UNSW-NB15-V2 , NF-BoT-IoT-V2', 'PROTOCOL', 0.16657182380704127, 1.280836256574538e-101], ['NF-UNSW-NB15-V2 , NF-CSE-CIC-IDS2018-V2', 'PROTOCOL', 0.17425695110258865, 2.6317873488904356e-110], ['NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2', 'PROTOCOL', 0.007685127295547378, 2.0890943325743977e-24]],[['NF-ToN-IoT-V2 , NF-UNSW-NB15-V2', 'L7_PROTO', 0.5815524055988588, 0.0], ['NF-ToN-IoT-V2 , NF-BoT-IoT-V2', 'L7_PROTO', 0.9886982748991158, 0.0], ['NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2', 'L7_PROTO', 0.987844592952798, 0.0], ['NF-UNSW-NB15-V2 , NF-BoT-IoT-V2', 'L7_PROTO', 0.5125472722968853, 0.0], ['NF-UNSW-NB15-V2 , NF-CSE-CIC-IDS2018-V2', 'L7_PROTO', 0.48441994247363374, 0.0], ['NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2', 'L7_PROTO', 0.029043913598867415, 0.0]],[['

In [41]:
flat_list = [element for sublist in results for element in sublist]

In [42]:
result_df = pd.DataFrame(data=flat_list, columns=['datasets', 'feature', 'stat', 'p_value']).sort_values(by='p_value', ascending=False)
result_df

,datasets,feature,stat,p_value
233,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
133,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_BYTES,0.000029,1.0
139,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_PKTS,0.000025,1.0
175,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",NUM_PKTS_512_TO_1024_BYTES,0.000142,1.0
230,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
...,...,...,...,...
87,"NF-UNSW-NB15-V2 , NF-BoT-IoT-V2",LONGEST_FLOW_PKT,0.762004,0.0
88,"NF-UNSW-NB15-V2 , NF-CSE-CIC-IDS2018-V2",LONGEST_FLOW_PKT,0.602813,0.0
89,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",LONGEST_FLOW_PKT,0.964699,0.0
90,"NF-ToN-IoT-V2 , NF-UNSW-NB15-V2",SHORTEST_FLOW_PKT,0.729276,0.0


In [47]:
result_df.to_csv("results_df",index=False)

In [4]:
result_df=pd.read_csv("results_df")
result_df

,datasets,feature,stat,p_value
0,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
1,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_BYTES,0.000029,1.0
2,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_PKTS,0.000025,1.0
3,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",NUM_PKTS_512_TO_1024_BYTES,0.000142,1.0
4,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
...,...,...,...,...
229,"NF-UNSW-NB15-V2 , NF-BoT-IoT-V2",LONGEST_FLOW_PKT,0.762004,0.0
230,"NF-UNSW-NB15-V2 , NF-CSE-CIC-IDS2018-V2",LONGEST_FLOW_PKT,0.602813,0.0
231,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",LONGEST_FLOW_PKT,0.964699,0.0
232,"NF-ToN-IoT-V2 , NF-UNSW-NB15-V2",SHORTEST_FLOW_PKT,0.729276,0.0


In [5]:
result_df[result_df.p_value>0.05]

,datasets,feature,stat,p_value
0,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
1,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_BYTES,0.000029,1.000000
2,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",RETRANSMITTED_OUT_PKTS,0.000025,1.000000
3,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",NUM_PKTS_512_TO_1024_BYTES,0.000142,1.000000
4,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
5,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
6,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_TYPE,0.000000,1.000000
7,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_TTL_ANSWER,0.000000,1.000000
8,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_ID,0.000000,1.000000
9,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",NUM_PKTS_1024_TO_1514_BYTES,0.000381,1.000000


In [11]:
result_df[result_df.p_value>0.05].groupby(["datasets"]).feature.count()

datasets
NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2      5
NF-ToN-IoT-V2 , NF-BoT-IoT-V2              5
NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2      2
NF-UNSW-NB15-V2 , NF-BoT-IoT-V2            2
NF-UNSW-NB15-V2 , NF-CSE-CIC-IDS2018-V2    2
Name: feature, dtype: int64

### DDoS

In [21]:
def evaluate_feature(feature):
    data= list(datasets.keys())
    result=[]
    for i in range(len(data)):
        if data[i] in "NF-UNSW-NB15-V2":
            continue
        for j in range(i+1,len(data)):
            if data[j] in "NF-UNSW-NB15-V2":
                continue
            # print(f"{data[i]} , {data[j]}")
            stat, p_value = kstest(datasets[data[i]][datasets[data[i]].Attack=="DDoS"][feature], datasets[data[j]][datasets[data[j]].Attack=="DDoS"][feature])
            # if(p_value>0.05):
            result.append([f"{data[i]} , {data[j]}", feature, stat,p_value])
    return result

In [22]:
from fastcore.parallel import *
from os import cpu_count
results = parallel(f=evaluate_feature, items=features, n_workers=cpu_count(), threadpool=True, progress=True)

In [29]:
flat_list = [element for sublist in results for element in sublist]

In [24]:
result_df = pd.DataFrame(data=flat_list, columns=['datasets', 'feature', 'stat', 'p_value']).sort_values(by='p_value', ascending=False)
result_df

,datasets,feature,stat,p_value
116,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
115,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
114,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
113,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_TTL_ANSWER,0.000000,1.0
110,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_TYPE,0.000000,1.0
...,...,...,...,...
34,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_OUT,0.385094,0.0
33,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",DURATION_OUT,0.710853,0.0
32,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_IN,0.939230,0.0
31,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_IN,0.316822,0.0


In [ ]:
result_df.to_csv("results_df_ddos",index=False)

In [ ]:
result_df=pd.read_csv("results_df_ddos")
result_df

,datasets,feature,stat,p_value
0,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
1,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
2,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",FTP_COMMAND_RET_CODE,0.000000,1.0
3,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_TTL_ANSWER,0.000000,1.0
4,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_TYPE,0.000000,1.0
...,...,...,...,...
112,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_OUT,0.385094,0.0
113,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",DURATION_OUT,0.710853,0.0
114,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_IN,0.939230,0.0
115,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DURATION_IN,0.316822,0.0


In [ ]:
result_df[result_df.p_value>0.05]

,datasets,feature,stat,p_value
0,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
1,"NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
2,"NF-ToN-IoT-V2 , NF-BoT-IoT-V2",FTP_COMMAND_RET_CODE,0.000000,1.000000
3,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_TTL_ANSWER,0.000000,1.000000
4,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_TYPE,0.000000,1.000000
5,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",DNS_QUERY_ID,0.000000,1.000000
6,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",RETRANSMITTED_OUT_BYTES,0.000076,1.000000
7,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",RETRANSMITTED_OUT_PKTS,0.000076,1.000000
8,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",NUM_PKTS_1024_TO_1514_BYTES,0.000002,1.000000
9,"NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2",RETRANSMITTED_IN_PKTS,0.000272,0.999995


In [ ]:
result_df[result_df.p_value>0.05].groupby(["datasets"]).feature.count()

datasets
NF-BoT-IoT-V2 , NF-CSE-CIC-IDS2018-V2    8
NF-ToN-IoT-V2 , NF-BoT-IoT-V2            1
NF-ToN-IoT-V2 , NF-CSE-CIC-IDS2018-V2    1
Name: feature, dtype: int64